In [172]:
# datetime per hour, fire, other details such as tree or metadata, polygon

# create geopandas
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from datetime import datetime, timedelta

# Define the bounding box (frame) for the grid
# (min_lon, min_lat, max_lon, max_lat)
bounding_box = (20.0, 34.0, 29.0, 42.0)  # Example for area of Greece

# Spacing between polygons (0.25 degrees)
spacing = 0.25
sh = spacing/2 # halved area

# Create the polygons grid within the bounding box
def create_grid(min_lon, min_lat, max_lon, max_lat, spacing):
    gids = []
    gid = 1
    polygons = []
    lons = []
    lats = []
    x = min_lon
    while x < max_lon:
        y = min_lat
        while y < max_lat:
            # Create a square polygon
            poly = Polygon([
                (x-sh, y-sh), 
                (x+sh, y-sh), 
                (x+sh, y+sh), 
                (x-sh, y+sh)
            ])
            gids.append(gid)
            polygons.append(poly)
            lons.append(x)
            lats.append(y)

            y += spacing
            gid += 1
        x += spacing
    return gids, polygons, lons, lats

# Generate the grid of polygons
gids, polygons, lons, lats = create_grid(*bounding_box, spacing)



In [173]:
print(len(gids))

1152


In [174]:
# Create a time range with hourly intervals from June 2023 to August 2023
start_time = datetime(2023, 6, 1, 0, 0)
end_time = datetime(2023, 8, 31, 22, 59)
start_time_range = pd.date_range(start_time, end_time, freq='1h', inclusive="left")
end_time_range = pd.date_range(datetime(2023, 6, 1, 1, 0), datetime(2023, 8, 31, 23, 59), freq="1h", inclusive="left")

print(start_time_range.__len__)

<bound method Index.__len__ of DatetimeIndex(['2023-06-01 00:00:00', '2023-06-01 01:00:00',
               '2023-06-01 02:00:00', '2023-06-01 03:00:00',
               '2023-06-01 04:00:00', '2023-06-01 05:00:00',
               '2023-06-01 06:00:00', '2023-06-01 07:00:00',
               '2023-06-01 08:00:00', '2023-06-01 09:00:00',
               ...
               '2023-08-31 13:00:00', '2023-08-31 14:00:00',
               '2023-08-31 15:00:00', '2023-08-31 16:00:00',
               '2023-08-31 17:00:00', '2023-08-31 18:00:00',
               '2023-08-31 19:00:00', '2023-08-31 20:00:00',
               '2023-08-31 21:00:00', '2023-08-31 22:00:00'],
              dtype='datetime64[ns]', length=2207, freq='H')>


In [175]:
# create polygon
rows = []

# Iterate over each polygon and each timestamp
for j in range(len(start_time_range)):
    for i, gid in enumerate(gids):
        rows.append({
            'id': gid,
            'start_time': start_time_range[j],
            'end_time': end_time_range[j],
            'lon': lons[i],  # Add longitude of bottom-left corner for reference
            'lat': lats[i],  # Add latitude of bottom-left corner for reference
        })

In [176]:
poly_rows = []

for i, zipped in enumerate(zip(gids, polygons)):
    gid, poly = zipped
    poly_rows.append({
        "id": gid,
        "geometry": poly
    })

In [177]:
short_df = pd.DataFrame(rows)
grid_gdf = gpd.GeoDataFrame(poly_rows, crs="EPSG:4326")

# Save to a file (GeoPackage or any format)
grid_gdf.to_file('polygons.gpkg', driver='GPKG')

# Print some of the data for verification
print(grid_gdf.head())
grid_gdf.describe()

   id                                           geometry
0   1  POLYGON ((19.87500 33.87500, 20.12500 33.87500...
1   2  POLYGON ((19.87500 34.12500, 20.12500 34.12500...
2   3  POLYGON ((19.87500 34.37500, 20.12500 34.37500...
3   4  POLYGON ((19.87500 34.62500, 20.12500 34.62500...
4   5  POLYGON ((19.87500 34.87500, 20.12500 34.87500...


,id
count,1152.000000
mean,576.500000
std,332.698061
min,1.000000
25%,288.750000
50%,576.500000
75%,864.250000
max,1152.000000


In [178]:
grid_gdf.to_json()

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"id": 1}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 33.875], [20.125, 33.875], [20.125, 34.125], [19.875, 34.125], [19.875, 33.875]]]}}, {"id": "1", "type": "Feature", "properties": {"id": 2}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 34.125], [20.125, 34.125], [20.125, 34.375], [19.875, 34.375], [19.875, 34.125]]]}}, {"id": "2", "type": "Feature", "properties": {"id": 3}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 34.375], [20.125, 34.375], [20.125, 34.625], [19.875, 34.625], [19.875, 34.375]]]}}, {"id": "3", "type": "Feature", "properties": {"id": 4}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 34.625], [20.125, 34.625], [20.125, 34.875], [19.875, 34.875], [19.875, 34.625]]]}}, {"id": "4", "type": "Feature", "properties": {"id": 5}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 34.875], [20.125, 34.875], [20.125, 35.125], [

In [179]:
short_df.head()
short_df.describe()

,id,lon,lat
count,2.542464e+06,2.542464e+06,2.542464e+06
mean,5.765000e+02,2.437500e+01,3.787500e+01
std,3.325537e+02,2.597074e+00,2.308274e+00
min,1.000000e+00,2.000000e+01,3.400000e+01
25%,2.887500e+02,2.218750e+01,3.593750e+01
50%,5.765000e+02,2.437500e+01,3.787500e+01
75%,8.642500e+02,2.656250e+01,3.981250e+01
max,1.152000e+03,2.875000e+01,4.175000e+01


In [180]:
# create target area to filter grid

geojson_path = 'world_HD.geo.json'

# Load the GeoJSON into a GeoDataFrame
gdf = gpd.read_file(geojson_path)

# Assuming your GeoJSON contains MultiPolygons or polygons, 
# you can filter for MultiPolygons if needed
multipolygons = gdf[gdf.geometry.type == 'MultiPolygon']

gdf.head()

gdf_greece = gdf[gdf['sovereignt'] == 'Greece']


In [181]:
gdf_greece

,featurecla,scalerank,labelrank,sovereignt,sov_a3,adm0_dif,level,type,tlc,admin,...,fclass_id,fclass_pl,fclass_gr,fclass_it,fclass_nl,fclass_se,fclass_bd,fclass_ua,filename,geometry
196,Admin-0 country,0,3,Greece,GRC,0,2,Sovereign country,1,Greece,...,None,None,None,None,None,None,None,None,GRC.geojson,"MULTIPOLYGON (((20.96526 40.84939, 21.11213 40..."


In [182]:
gdf_greece.to_json()
grid_gdf

,id,geometry
0,1,"POLYGON ((19.87500 33.87500, 20.12500 33.87500..."
1,2,"POLYGON ((19.87500 34.12500, 20.12500 34.12500..."
2,3,"POLYGON ((19.87500 34.37500, 20.12500 34.37500..."
3,4,"POLYGON ((19.87500 34.62500, 20.12500 34.62500..."
4,5,"POLYGON ((19.87500 34.87500, 20.12500 34.87500..."
...,...,...
1147,1148,"POLYGON ((28.62500 40.62500, 28.87500 40.62500..."
1148,1149,"POLYGON ((28.62500 40.87500, 28.87500 40.87500..."
1149,1150,"POLYGON ((28.62500 41.12500, 28.87500 41.12500..."
1150,1151,"POLYGON ((28.62500 41.37500, 28.87500 41.37500..."


In [183]:
multipolygon = gdf_greece.geometry.iloc[0]

# Filter the polygons in gdf_polygons that intersect with the MultiPolygon
filtered_polygons = grid_gdf[grid_gdf.intersects(multipolygon)]

# Print the filtered GeoDataFrame
print(filtered_polygons)
filtered_polygons.to_json()

        id                                           geometry
21      22  POLYGON ((19.87500 39.12500, 20.12500 39.12500...
22      23  POLYGON ((19.87500 39.37500, 20.12500 39.37500...
23      24  POLYGON ((19.87500 39.62500, 20.12500 39.62500...
49      50  POLYGON ((20.12500 38.12500, 20.37500 38.12500...
53      54  POLYGON ((20.12500 39.12500, 20.37500 39.12500...
...    ...                                                ...
1032  1033  POLYGON ((27.87500 35.87500, 28.12500 35.87500...
1033  1034  POLYGON ((27.87500 36.12500, 28.12500 36.12500...
1034  1035  POLYGON ((27.87500 36.37500, 28.12500 36.37500...
1065  1066  POLYGON ((28.12500 36.12500, 28.37500 36.12500...
1066  1067  POLYGON ((28.12500 36.37500, 28.37500 36.37500...

[440 rows x 2 columns]


'{"type": "FeatureCollection", "features": [{"id": "21", "type": "Feature", "properties": {"id": 22}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 39.125], [20.125, 39.125], [20.125, 39.375], [19.875, 39.375], [19.875, 39.125]]]}}, {"id": "22", "type": "Feature", "properties": {"id": 23}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 39.375], [20.125, 39.375], [20.125, 39.625], [19.875, 39.625], [19.875, 39.375]]]}}, {"id": "23", "type": "Feature", "properties": {"id": 24}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 39.625], [20.125, 39.625], [20.125, 39.875], [19.875, 39.875], [19.875, 39.625]]]}}, {"id": "49", "type": "Feature", "properties": {"id": 50}, "geometry": {"type": "Polygon", "coordinates": [[[20.125, 38.125], [20.375, 38.125], [20.375, 38.375], [20.125, 38.375], [20.125, 38.125]]]}}, {"id": "53", "type": "Feature", "properties": {"id": 54}, "geometry": {"type": "Polygon", "coordinates": [[[20.125, 39.125], [20.375, 39.125], [20.375, 

In [184]:
# only have time and polygons that exist in Greece

# create polygon
rows = []

# Iterate over each polygon and each timestamp
for j in range(len(start_time_range)):
    for i, frow in filtered_polygons.iterrows():
        rows.append({
            'id': frow["id"],
            'start_time': start_time_range[j],
            'end_time': end_time_range[j],
            'lon': lons[i],  # Add longitude of bottom-left corner for reference
            'lat': lats[i],  # Add latitude of bottom-left corner for reference
        })

hour_df = pd.DataFrame(rows)
hour_df.head()

,id,start_time,end_time,lon,lat
0,22,2023-06-01,2023-06-01 01:00:00,20.00,39.25
1,23,2023-06-01,2023-06-01 01:00:00,20.00,39.50
2,24,2023-06-01,2023-06-01 01:00:00,20.00,39.75
3,50,2023-06-01,2023-06-01 01:00:00,20.25,38.25
4,54,2023-06-01,2023-06-01 01:00:00,20.25,39.25


In [185]:
# Load the GeoJSON and grid GeoDataFrames
fire_gdf = gpd.read_file('transformed_file.geojson')
filtered_grid = filtered_polygons

matches = []

for i, grid_poly in fire_gdf.iterrows():
    for j, comp_poly in filtered_grid.iterrows():
        # Step 3: Check for intersections
        if grid_poly.geometry.intersects(comp_poly.geometry):
            matches.append({"fire": grid_poly["id"], "polygon":comp_poly["id"]})


In [186]:
matches

[{'fire': 14843, 'polygon': 912},
 {'fire': 14389, 'polygon': 210},
 {'fire': 14329, 'polygon': 311},
 {'fire': 15314, 'polygon': 113},
 {'fire': 15002, 'polygon': 444},
 {'fire': 14324, 'polygon': 444},
 {'fire': 14946, 'polygon': 444},
 {'fire': 14676, 'polygon': 665},
 {'fire': 15324, 'polygon': 665},
 {'fire': 15103, 'polygon': 537},
 {'fire': 14219, 'polygon': 590},
 {'fire': 15269, 'polygon': 790},
 {'fire': 15280, 'polygon': 790},
 {'fire': 15422, 'polygon': 909},
 {'fire': 15437, 'polygon': 614},
 {'fire': 15471, 'polygon': 342},
 {'fire': 15492, 'polygon': 790},
 {'fire': 15496, 'polygon': 279},
 {'fire': 15665, 'polygon': 531},
 {'fire': 15819, 'polygon': 148},
 {'fire': 15896, 'polygon': 121},
 {'fire': 15911, 'polygon': 339},
 {'fire': 16011, 'polygon': 309},
 {'fire': 16167, 'polygon': 790},
 {'fire': 16195, 'polygon': 790},
 {'fire': 16211, 'polygon': 86},
 {'fire': 16252, 'polygon': 402},
 {'fire': 16275, 'polygon': 510},
 {'fire': 16361, 'polygon': 467},
 {'fire': 16361

In [187]:
print(len(matches))
filtered_grid.loc[filtered_grid['id'] == 594,'geometry'].to_json()

1228


'{"type": "FeatureCollection", "features": [{"id": "593", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[24.375, 38.125], [24.625, 38.125], [24.625, 38.375], [24.375, 38.375], [24.375, 38.125]]]}, "bbox": [24.375, 38.125, 24.625, 38.375]}], "bbox": [24.375, 38.125, 24.625, 38.375]}'

In [188]:
fire_gdf.loc[fire_gdf['id'] == 223671,'geometry'].to_json()

'{"type": "FeatureCollection", "features": [{"id": "888", "type": "Feature", "properties": {}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[24.58939335304745, 38.140701237777805], [24.58933579767129, 38.14075441671835], [24.589301790027775, 38.14082761608412], [24.58929133017141, 38.140920835884245], [24.589304418219584, 38.141034076132165], [24.5893410542942, 38.141167336790154], [24.589401238615526, 38.1413206178284], [24.589484971419537, 38.141493919164326], [24.589592252971556, 38.14168724072473], [24.58972308362551, 38.14190058239986], [24.589841902545732, 38.14209646553153], [24.58994870951788, 38.14227489018385], [24.590043504326456, 38.142435856417926], [24.59012628679034, 38.1425793642956], [24.590197056754963, 38.14270541384387], [24.59025581407241, 38.14281400512228], [24.590302558630583, 38.1429051381449], [24.590337290324026, 38.14297881295837], [24.590368919081694, 38.14304600656862], [24.590397444883585, 38.143106718990005], [24.590422867706028, 38.14316095020

In [189]:
fire_gdf.head()

,id,initialdate,finaldate,area_ha,iso2,iso3,country,admlvl1,admlvl2,admlvl3,...,other_natural_percent,agriculture_percent,artificial_percent,other_percent,natura2k_percent,noneu,updated,area_code,eu_area,geometry
0,14843,2018-09-09,2018-09-10 00:00:00+00:00,147,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Βόρειο Αιγαίο,"Ικαρία, Σάμος",...,0.000000,35.135135,0.000000,0.0,0.000000,False,2022-01-26 12:15:03.089,EU,EU,"MULTIPOLYGON (((26.920 37.766, 26.920 37.766, ..."
1,14389,2018-09-21,2018-09-21 00:00:00+00:00,75,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Δυτική Ελλάδα,Αχαΐα,...,97.297297,2.702703,0.000000,0.0,66.535296,False,2022-01-26 12:15:03.089,EU,EU,"MULTIPOLYGON (((21.408 38.143, 21.408 38.143, ..."
2,14329,2018-09-23,2018-09-23 00:00:00+00:00,192,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Θεσσαλία,Λάρισα,...,54.736842,45.263158,0.000000,0.0,85.111159,False,2022-01-26 12:15:03.089,EU,EU,"MULTIPOLYGON (((22.249 39.476, 22.249 39.476, ..."
3,15314,2018-09-27,2018-09-27 00:00:00+00:00,270,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Ιόνια Νησιά,"Ιθάκη, Κεφαλληνία",...,18.888889,67.037037,0.740741,0.0,0.000000,False,2022-01-26 12:15:03.089,EU,EU,"MULTIPOLYGON (((20.751 38.074, 20.751 38.074, ..."
4,15002,2018-10-09,2018-10-09 00:00:00+00:00,51,EL,GRC,Ελλάδα,Βόρεια Ελλάδα,Κεντρική Μακεδονία,Θεσσαλονίκη,...,96.000000,0.000000,0.000000,0.0,100.000000,False,2022-01-26 12:15:03.089,EU,EU,"MULTIPOLYGON (((23.324 40.731, 23.324 40.731, ..."


In [190]:
fire_gdf['start_date'] = pd.to_datetime(fire_gdf['initialdate'], format='mixed')
fire_gdf['end_date'] = pd.to_datetime(fire_gdf['finaldate'], format='mixed')
# Step 3: Filter the DataFrame based on the date column
filtered_fire = fire_gdf[(fire_gdf['start_date'] >= datetime(2023, 6, 1, 0, 0)) & (fire_gdf['start_date'] <= datetime(2023, 8, 31, 23, 59))]

In [212]:
def hour_rounder(t):
    # Rounds to nearest hour 
    return t.replace(second=0, microsecond=0, minute=0, hour=t.hour)

df_rows = pd.DataFrame(rows)
df_rows['fire'] = 0

matches = pd.DataFrame(matches)

df_rows['start_time'] = pd.to_datetime(df_rows['start_time'], format='mixed').dt.tz_localize(None)
df_rows['end_time'] = pd.to_datetime(df_rows['end_time'], format='mixed').dt.tz_localize(None)
filtered_fire['start_date'] = pd.to_datetime(filtered_fire['start_date'], format='mixed').dt.tz_localize(None)
filtered_fire['end_date'] = pd.to_datetime(filtered_fire['end_date'], format='mixed').dt.tz_localize(None)

/Users/paulafrindte/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/paulafrindte/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [213]:
# Update "fire" indicator in df_rows to represent fires and the 24/72 hours before
for fire_index, fire_row in filtered_fire.iterrows(): 
    current_date = hour_rounder(fire_row['start_date'])
    
    matching_polygons = list(matches.loc[matches['fire']==fire_row['id'],'polygon'])
    
    while current_date < hour_rounder(fire_row['end_date']) + timedelta(hours=1):
        # 100% risk during a fire
        df_rows.loc[(df_rows['id'].isin(matching_polygons))&(df_rows['start_time']==current_date),'fire'] = 1
        # 80% risk on 3 days before the fire
        df_rows.loc[(df_rows['id'].isin(matching_polygons))&(df_rows['start_time']<current_date)&(df_rows['start_time']>=current_date-timedelta(hours=72)),'fire'] = 0.8
        # 95% risk on the day / 24h of the fire
        df_rows.loc[(df_rows['id'].isin(matching_polygons))&(df_rows['start_time']<current_date)&(df_rows['start_time']>=current_date-timedelta(hours=24)),'fire'] = 0.95
        
        
        #print(current_date)
        current_date += timedelta(hours=1)


[717]
2023-07-14 10:00:00
2023-07-14 11:00:00
2023-07-14 12:00:00
2023-07-14 13:00:00
2023-07-14 14:00:00
2023-07-14 15:00:00
2023-07-14 16:00:00
2023-07-14 17:00:00
2023-07-14 18:00:00
2023-07-14 19:00:00
2023-07-14 20:00:00
2023-07-14 21:00:00
2023-07-14 22:00:00
2023-07-14 23:00:00
2023-07-15 00:00:00
2023-07-15 01:00:00
2023-07-15 02:00:00
2023-07-15 03:00:00
2023-07-15 04:00:00
2023-07-15 05:00:00
2023-07-15 06:00:00
2023-07-15 07:00:00
2023-07-15 08:00:00
2023-07-15 09:00:00
2023-07-15 10:00:00
2023-07-15 11:00:00
2023-07-15 12:00:00
2023-07-15 13:00:00
2023-07-15 14:00:00
2023-07-15 15:00:00
2023-07-15 16:00:00
2023-07-15 17:00:00
2023-07-15 18:00:00
2023-07-15 19:00:00
2023-07-15 20:00:00
2023-07-15 21:00:00
2023-07-15 22:00:00
2023-07-15 23:00:00
2023-07-16 00:00:00
[717]
2023-07-14 19:00:00
2023-07-14 20:00:00
2023-07-14 21:00:00
2023-07-14 22:00:00
2023-07-14 23:00:00
2023-07-15 00:00:00
[499]
2023-07-15 12:00:00
[498]
2023-07-06 10:00:00
2023-07-06 11:00:00
2023-07-06 12:00

2023-07-17 17:00:00
2023-07-17 18:00:00
2023-07-17 19:00:00
2023-07-17 20:00:00
2023-07-17 21:00:00
2023-07-17 22:00:00
2023-07-17 23:00:00
2023-07-18 00:00:00
2023-07-18 01:00:00
2023-07-18 02:00:00
2023-07-18 03:00:00
2023-07-18 04:00:00
2023-07-18 05:00:00
2023-07-18 06:00:00
2023-07-18 07:00:00
2023-07-18 08:00:00
2023-07-18 09:00:00
2023-07-18 10:00:00
2023-07-18 11:00:00
[331]
2023-07-14 12:00:00
[333]
2023-07-19 12:00:00
2023-07-19 13:00:00
2023-07-19 14:00:00
2023-07-19 15:00:00
2023-07-19 16:00:00
2023-07-19 17:00:00
2023-07-19 18:00:00
2023-07-19 19:00:00
2023-07-19 20:00:00
2023-07-19 21:00:00
2023-07-19 22:00:00
2023-07-19 23:00:00
2023-07-20 00:00:00
2023-07-20 01:00:00
2023-07-20 02:00:00
2023-07-20 03:00:00
2023-07-20 04:00:00
2023-07-20 05:00:00
2023-07-20 06:00:00
2023-07-20 07:00:00
2023-07-20 08:00:00
2023-07-20 09:00:00
2023-07-20 10:00:00
2023-07-20 11:00:00
2023-07-20 12:00:00
2023-07-20 13:00:00
2023-07-20 14:00:00
2023-07-20 15:00:00
2023-07-20 16:00:00
2023-07-

2023-08-23 10:00:00
2023-08-23 11:00:00
2023-08-23 12:00:00
2023-08-23 13:00:00
2023-08-23 14:00:00
2023-08-23 15:00:00
2023-08-23 16:00:00
2023-08-23 17:00:00
2023-08-23 18:00:00
2023-08-23 19:00:00
2023-08-23 20:00:00
2023-08-23 21:00:00
2023-08-23 22:00:00
2023-08-23 23:00:00
2023-08-24 00:00:00
2023-08-24 01:00:00
2023-08-24 02:00:00
2023-08-24 03:00:00
2023-08-24 04:00:00
2023-08-24 05:00:00
2023-08-24 06:00:00
2023-08-24 07:00:00
2023-08-24 08:00:00
2023-08-24 09:00:00
2023-08-24 10:00:00
2023-08-24 11:00:00
2023-08-24 12:00:00
2023-08-24 13:00:00
2023-08-24 14:00:00
2023-08-24 15:00:00
2023-08-24 16:00:00
2023-08-24 17:00:00
2023-08-24 18:00:00
2023-08-24 19:00:00
2023-08-24 20:00:00
2023-08-24 21:00:00
2023-08-24 22:00:00
2023-08-24 23:00:00
2023-08-25 00:00:00
2023-08-25 01:00:00
2023-08-25 02:00:00
2023-08-25 03:00:00
2023-08-25 04:00:00
2023-08-25 05:00:00
2023-08-25 06:00:00
2023-08-25 07:00:00
2023-08-25 08:00:00
2023-08-25 09:00:00
2023-08-25 10:00:00
2023-08-25 11:00:00


2023-09-09 15:00:00
2023-09-09 16:00:00
2023-09-09 17:00:00
2023-09-09 18:00:00
2023-09-09 19:00:00
2023-09-09 20:00:00
2023-09-09 21:00:00
2023-09-09 22:00:00
2023-09-09 23:00:00
2023-09-10 00:00:00
2023-09-10 01:00:00
2023-09-10 02:00:00
2023-09-10 03:00:00
2023-09-10 04:00:00
2023-09-10 05:00:00
2023-09-10 06:00:00
2023-09-10 07:00:00
2023-09-10 08:00:00
2023-09-10 09:00:00
2023-09-10 10:00:00
2023-09-10 11:00:00
2023-09-10 12:00:00
2023-09-10 13:00:00
2023-09-10 14:00:00
2023-09-10 15:00:00
2023-09-10 16:00:00
2023-09-10 17:00:00
2023-09-10 18:00:00
2023-09-10 19:00:00
2023-09-10 20:00:00
2023-09-10 21:00:00
2023-09-10 22:00:00
2023-09-10 23:00:00
2023-09-11 00:00:00
[734]
2023-08-21 11:00:00
2023-08-21 12:00:00
2023-08-21 13:00:00
2023-08-21 14:00:00
2023-08-21 15:00:00
2023-08-21 16:00:00
2023-08-21 17:00:00
2023-08-21 18:00:00
2023-08-21 19:00:00
2023-08-21 20:00:00
2023-08-21 21:00:00
2023-08-21 22:00:00
2023-08-21 23:00:00
2023-08-22 00:00:00
2023-08-22 01:00:00
2023-08-22 02:

2023-08-22 17:00:00
2023-08-22 18:00:00
2023-08-22 19:00:00
2023-08-22 20:00:00
2023-08-22 21:00:00
2023-08-22 22:00:00
2023-08-22 23:00:00
2023-08-23 00:00:00
2023-08-23 01:00:00
2023-08-23 02:00:00
2023-08-23 03:00:00
2023-08-23 04:00:00
2023-08-23 05:00:00
2023-08-23 06:00:00
2023-08-23 07:00:00
2023-08-23 08:00:00
2023-08-23 09:00:00
2023-08-23 10:00:00
2023-08-23 11:00:00
2023-08-23 12:00:00
2023-08-23 13:00:00
2023-08-23 14:00:00
2023-08-23 15:00:00
2023-08-23 16:00:00
2023-08-23 17:00:00
2023-08-23 18:00:00
2023-08-23 19:00:00
2023-08-23 20:00:00
2023-08-23 21:00:00
2023-08-23 22:00:00
2023-08-23 23:00:00
2023-08-24 00:00:00
2023-08-24 01:00:00
2023-08-24 02:00:00
2023-08-24 03:00:00
2023-08-24 04:00:00
2023-08-24 05:00:00
2023-08-24 06:00:00
2023-08-24 07:00:00
2023-08-24 08:00:00
2023-08-24 09:00:00
2023-08-24 10:00:00
2023-08-24 11:00:00
[370]
2023-08-22 00:00:00
2023-08-22 01:00:00
2023-08-22 02:00:00
2023-08-22 03:00:00
2023-08-22 04:00:00
2023-08-22 05:00:00
2023-08-22 06:

2023-07-24 13:00:00
2023-07-24 14:00:00
2023-07-24 15:00:00
2023-07-24 16:00:00
2023-07-24 17:00:00
2023-07-24 18:00:00
2023-07-24 19:00:00
2023-07-24 20:00:00
2023-07-24 21:00:00
2023-07-24 22:00:00
2023-07-24 23:00:00
2023-07-25 00:00:00
2023-07-25 01:00:00
2023-07-25 02:00:00
2023-07-25 03:00:00
2023-07-25 04:00:00
2023-07-25 05:00:00
2023-07-25 06:00:00
2023-07-25 07:00:00
2023-07-25 08:00:00
2023-07-25 09:00:00
2023-07-25 10:00:00
2023-07-25 11:00:00
2023-07-25 12:00:00
2023-07-25 13:00:00
2023-07-25 14:00:00
2023-07-25 15:00:00
2023-07-25 16:00:00
2023-07-25 17:00:00
2023-07-25 18:00:00
2023-07-25 19:00:00
2023-07-25 20:00:00
2023-07-25 21:00:00
2023-07-25 22:00:00
2023-07-25 23:00:00
2023-07-26 00:00:00
[593]
2023-07-23 10:00:00
2023-07-23 11:00:00
2023-07-23 12:00:00
2023-07-23 13:00:00
2023-07-23 14:00:00
2023-07-23 15:00:00
2023-07-23 16:00:00
2023-07-23 17:00:00
2023-07-23 18:00:00
2023-07-23 19:00:00
2023-07-23 20:00:00
2023-07-23 21:00:00
2023-07-23 22:00:00
2023-07-23 23:

In [215]:
pd.set_option('display.max_rows', 100)
df_rows

,id,start_time,end_time,lon,lat,fire
0,22,2023-06-01 00:00:00,2023-06-01 01:00:00,20.00,39.25,0.0
1,23,2023-06-01 00:00:00,2023-06-01 01:00:00,20.00,39.50,0.0
2,24,2023-06-01 00:00:00,2023-06-01 01:00:00,20.00,39.75,0.0
3,50,2023-06-01 00:00:00,2023-06-01 01:00:00,20.25,38.25,0.0
4,54,2023-06-01 00:00:00,2023-06-01 01:00:00,20.25,39.25,0.0
...,...,...,...,...,...,...
971075,1033,2023-08-31 22:00:00,2023-08-31 23:00:00,28.00,36.00,0.0
971076,1034,2023-08-31 22:00:00,2023-08-31 23:00:00,28.00,36.25,0.0
971077,1035,2023-08-31 22:00:00,2023-08-31 23:00:00,28.00,36.50,0.0
971078,1066,2023-08-31 22:00:00,2023-08-31 23:00:00,28.25,36.25,0.0


In [220]:
df_rows.loc[(df_rows['id']==717)&(df_rows['start_time']>='2023-07-13'),:]
#pd.set_option('display.max_rows', None)
#df_rows.loc[(df_rows['start_time']=='2023-07-25 00:00:00'),:]

,id,start_time,end_time,lon,lat,fire
443866,717,2023-07-13 00:00:00,2023-07-13 01:00:00,25.5,37.0,0.8
444306,717,2023-07-13 01:00:00,2023-07-13 02:00:00,25.5,37.0,0.8
444746,717,2023-07-13 02:00:00,2023-07-13 03:00:00,25.5,37.0,0.8
445186,717,2023-07-13 03:00:00,2023-07-13 04:00:00,25.5,37.0,0.8
445626,717,2023-07-13 04:00:00,2023-07-13 05:00:00,25.5,37.0,0.8
...,...,...,...,...,...,...
969226,717,2023-08-31 18:00:00,2023-08-31 19:00:00,25.5,37.0,0.0
969666,717,2023-08-31 19:00:00,2023-08-31 20:00:00,25.5,37.0,0.0
970106,717,2023-08-31 20:00:00,2023-08-31 21:00:00,25.5,37.0,0.0
970546,717,2023-08-31 21:00:00,2023-08-31 22:00:00,25.5,37.0,0.0


In [221]:
df_rows.to_csv('fire_predictor.csv')

In [206]:
df_rows.loc[df_rows['fire']==True,:]

,id,start_time,end_time,lon,lat,fire
374234,498,2023-07-06 10:00:00,2023-07-06 11:00:00,23.75,38.25,True
374674,498,2023-07-06 11:00:00,2023-07-06 12:00:00,23.75,38.25,True
375114,498,2023-07-06 12:00:00,2023-07-06 13:00:00,23.75,38.25,True
378652,530,2023-07-06 20:00:00,2023-07-06 21:00:00,24.00,38.25,True
378653,531,2023-07-06 20:00:00,2023-07-06 21:00:00,24.00,38.50,True
379092,530,2023-07-06 21:00:00,2023-07-06 22:00:00,24.00,38.25,True
379093,531,2023-07-06 21:00:00,2023-07-06 22:00:00,24.00,38.50,True
379532,530,2023-07-06 22:00:00,2023-07-06 23:00:00,24.00,38.25,True
379533,531,2023-07-06 22:00:00,2023-07-06 23:00:00,24.00,38.50,True
379972,530,2023-07-06 23:00:00,2023-07-07 00:00:00,24.00,38.25,True


In [195]:
filtered_fire

,id,initialdate,finaldate,area_ha,iso2,iso3,country,admlvl1,admlvl2,admlvl3,...,artificial_percent,other_percent,natura2k_percent,noneu,updated,area_code,eu_area,geometry,start_date,end_date
780,217086,2023-07-14 10:38:00.000,2023-07-16 00:20:00+00:00,284,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Νότιο Αιγαίο,"Άνδρος, Θήρα, Κέα, Μήλος, Μύκονος, Νάξος, Πάρο...",...,0.0,0.0,0.003212,False,2023-07-17 10:36:01.365,EU,EU,"MULTIPOLYGON (((25.527 37.109, 25.526 37.109, ...",2023-07-14 10:38:00.000,2023-07-16 00:20:00.000
781,217136,2023-07-14 19:45:00.000,2023-07-15 00:37:00+00:00,2,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Νότιο Αιγαίο,"Άνδρος, Θήρα, Κέα, Μήλος, Μύκονος, Νάξος, Πάρο...",...,0.0,0.0,0.000000,False,2023-07-17 10:38:00.377,EU,EU,"MULTIPOLYGON (((25.515 37.090, 25.515 37.090, ...",2023-07-14 19:45:00.000,2023-07-15 00:37:00.000
789,217258,2023-07-15 12:00:00.000,2023-07-15 12:00:00+00:00,1,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Στερεά Ελλάδα,Εύβοια,...,0.0,0.0,0.000000,False,2023-07-20 07:49:25.961,EU,EU,"MULTIPOLYGON (((23.752 38.436, 23.752 38.436, ...",2023-07-15 12:00:00.000,2023-07-15 12:00:00.000
794,216949,2023-07-06 10:38:00.000,2023-07-06 12:17:00+00:00,12,EL,GRC,Ελλάδα,Αττική,Aττική,Ανατολική Αττική,...,0.0,0.0,0.000000,False,2024-01-25 15:19:41.600,EU,EU,"MULTIPOLYGON (((23.836 38.254, 23.836 38.254, ...",2023-07-06 10:38:00.000,2023-07-06 12:17:00.000
804,217260,2023-07-18 11:02:00.000,2023-07-18 11:02:00+00:00,2,EL,GRC,Ελλάδα,Αττική,Aττική,Δυτικός Τομέας Αθηνών,...,0.0,0.0,0.000000,False,2023-07-20 08:04:20.861,EU,EU,"MULTIPOLYGON (((23.628 38.020, 23.628 38.020, ...",2023-07-18 11:02:00.000,2023-07-18 11:02:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,217565,2023-07-25 00:03:00.000,2023-07-25 00:52:00+00:00,46,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Νότιο Αιγαίο,"Κάλυμνος, Κάρπαθος – Ηρωική Νήσος Κάσος, Κως, ...",...,0.0,0.0,0.000000,False,2023-07-31 09:55:02.799,EU,EU,"MULTIPOLYGON (((28.061 36.189, 28.060 36.189, ...",2023-07-25 00:03:00.000,2023-07-25 00:52:00.000
929,217993,2023-07-27 20:23:00.000,2023-07-28 01:34:00+00:00,97,EL,GRC,Ελλάδα,Βόρεια Ελλάδα,Ήπειρος,"Άρτα, Πρέβεζα",...,0.0,0.0,0.000000,False,2023-08-01 09:09:43.452,EU,EU,"MULTIPOLYGON (((20.881 39.257, 20.881 39.258, ...",2023-07-27 20:23:00.000,2023-07-28 01:34:00.000
930,217994,2023-08-01 00:00:05.436,2023-08-01 11:00:14.523000+00:00,7,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Ιόνια Νησιά,"Ιθάκη, Κεφαλληνία",...,0.0,0.0,0.000000,False,2023-08-01 10:00:24.440,EU,EU,"MULTIPOLYGON (((20.551 38.112, 20.550 38.111, ...",2023-08-01 00:00:05.436,2023-08-01 11:00:14.523
931,217941,2023-07-30 23:48:00.000,2023-07-31 00:37:00+00:00,87,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Βόρειο Αιγαίο,Χίος,...,0.0,0.0,99.933976,False,2023-08-01 10:06:37.534,EU,EU,"MULTIPOLYGON (((25.993 38.469, 25.993 38.469, ...",2023-07-30 23:48:00.000,2023-07-31 00:37:00.000


In [91]:
df_rows

,id,start_time,end_time,lon,lat,fire
0,22,2023-06-01 00:00:00,2023-06-01 01:00:00,20.00,39.25,False
1,23,2023-06-01 00:00:00,2023-06-01 01:00:00,20.00,39.50,False
2,24,2023-06-01 00:00:00,2023-06-01 01:00:00,20.00,39.75,False
3,50,2023-06-01 00:00:00,2023-06-01 01:00:00,20.25,38.25,False
4,54,2023-06-01 00:00:00,2023-06-01 01:00:00,20.25,39.25,False
...,...,...,...,...,...,...
73475,1033,2023-06-07 22:00:00,2023-06-07 23:00:00,28.00,36.00,False
73476,1034,2023-06-07 22:00:00,2023-06-07 23:00:00,28.00,36.25,False
73477,1035,2023-06-07 22:00:00,2023-06-07 23:00:00,28.00,36.50,False
73478,1066,2023-06-07 22:00:00,2023-06-07 23:00:00,28.25,36.25,False


In [83]:
filtered_grid

,id,geometry
21,22,"POLYGON ((19.87500 39.12500, 20.12500 39.12500..."
22,23,"POLYGON ((19.87500 39.37500, 20.12500 39.37500..."
23,24,"POLYGON ((19.87500 39.62500, 20.12500 39.62500..."
49,50,"POLYGON ((20.12500 38.12500, 20.37500 38.12500..."
53,54,"POLYGON ((20.12500 39.12500, 20.37500 39.12500..."
...,...,...
1032,1033,"POLYGON ((27.87500 35.87500, 28.12500 35.87500..."
1033,1034,"POLYGON ((27.87500 36.12500, 28.12500 36.12500..."
1034,1035,"POLYGON ((27.87500 36.37500, 28.12500 36.37500..."
1065,1066,"POLYGON ((28.12500 36.12500, 28.37500 36.12500..."


In [111]:
matches

,fire,polygon
0,14843,912
1,14389,210
2,14329,311
3,15314,113
4,15002,444
...,...,...
1223,239811,219
1224,241089,332
1225,240373,333
1226,241107,275
